# ETL Pipeline for Pre-Processing the Files

## THE CODE BELOW IS FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
# Uncomment the line below, if you want to see the files founded
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Creating and loading the cassandra tables. 

## Now we can work with the file <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Initializing the session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating the keyspace that will contains all of the tables related to the song playlist dataset
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
# Setting keyspace we created above
try:
    session.set_keyspace('project_udacity')
except Exception as e:
    print(e)

### Creating the tables on Cassandra

In [8]:
## The focus here, in this create table, is to look for the artist and his song title and length
## So then, we need to make a unique id with session id and item in session, to identify the artist, his respective song and length for our analysis

query = "CREATE TABLE IF NOT EXISTS song_playlist_artists"
query = query + "( sessionId bigint, itemInSession bigint, userId bigint, \
                  artist text, length float, song text, \
                  PRIMARY KEY ( sessionId, itemInSession ) )"


## The focus here, in this create table, is to look for the name of the artist and the user that listened to it in his session
## So then, we need to make a unique id with user id and session id, to identify the users who listened to a respective artist in his session

query2 = "CREATE TABLE IF NOT EXISTS song_playlist_sessions"
query2 = query2 + "( userId bigint, sessionId bigint, itemInSession bigint, \
                    artist text, firstName text, lastName text,  \
                    song text, PRIMARY KEY ( ( userId, sessionId ), itemInSession ) )"

## The focus here, in this create table, is to look for the name of the song and who listened
## So then, we need to make a unique id with song and user id, to identify what song the users listened to

query3 = "CREATE TABLE IF NOT EXISTS song_playlist_users"
query3 = query3 + "( song text, userId bigint, firstName text, lastName text,  \
                    PRIMARY KEY ( song, userId ) )"

queries = [query, query2, query3]

for qry in queries:
    try:
        session.execute(qry)
    except Exception as e:
        print(e)


In [ ]:
#Loading the event datafile, to load the cassandra tables

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Inserting the data into the cassandra tables - loading the variables
        query = "INSERT INTO song_playlist_artists ( sessionId, itemInSession, userId, artist, length, song )"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        
        query2 = "INSERT INTO song_playlist_sessions ( userId, sessionId, itemInSession, artist, firstName, lastName, song )"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        query3 = "INSERT INTO song_playlist_users ( song, userId, firstName, lastName )"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        
## Assigning the values for each column in the INSERT statements
        session.execute(query, ( int(line[8]), int(line[3]), int(line[10]), line[0], float(line[5]), line[9] ) )
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9] ) )
        session.execute(query3, (line[9], int(line[10]), line[1], line[4] ))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Querying the table we created above, with the artist and his song title/length

query = "SELECT artist, song, length FROM song_playlist_artists WHERE sessionId = 338 and itemInSession = 4"

#Initializing the pandas DF
df = pd.DataFrame()

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    df = df.append( { 'artist': row.artist, 'song': row.song, 'length': row.length }, ignore_index=True )

#Showing a head description of the query results we load into this DF

df.head()

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## Query to find the name of artist and the user who listened to in his session

query = "SELECT artist, song, firstname, lastname FROM song_playlist_sessions WHERE userid = 10 AND sessionid = 182 order by itemInSession"

#Initializing the pandas DF
df = pd.DataFrame()

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    df = df.append( { 'artist': row.artist, 'song': row.song, 'firstname': row.firstname, 'lastname': row.lastname }, ignore_index=True )

#Showing a head description of the query results we load into this DF

df.head()
                    

In [18]:
## Query to find who listened to the song 'All Hands Against His Own'

query = "SELECT firstname, lastname FROM song_playlist_users WHERE song = 'All Hands Against His Own'"
#Initializing the pandas DF
df = pd.DataFrame()

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    df = df.append( { 'firstname': row.firstname, 'lastname': row.lastname }, ignore_index=True )

#Showing a head description of the query results we load into this DF

df.head()
                    

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [ ]:
## Drop the tables, before closing the sessions

In [21]:
drop_tables = ['song_playlist_artists', 'song_playlist_sessions', 'song_playlist_users']

for drop in drop_tables:
    try:
        session.execute('drop table {} '.format(drop))
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()